In [13]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer, WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import pickle

In [14]:
# import dataset to train models
train = pd.read_csv('../Data/train.txt', sep=';', names=['content', 'sentiment'])

test = pd.read_csv('../Data/test.txt', sep=';', names=['content', 'sentiment'])

In [15]:
data = pd.read_csv('../Data/val.txt', sep=';', names=['content', 'sentiment'])
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18000 entries, 0 to 17999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   content    18000 non-null  object
 1   sentiment  18000 non-null  object
dtypes: object(2)
memory usage: 140.7+ KB


In [16]:
# assign each sentiment to a numeric value
vals_to_replace = {'sadness': 1, 'love': 5, 'anger': 0, 'joy': 4, 'fear': 2, 'surprise': 3}
data['sent_num'] = data.sentiment.map(vals_to_replace)

In [17]:
def stopword_del(sentence):
    stopwords_list = stopwords.words("english")
    word_tokens_test = word_tokenize(sentence.lower())
    tokens_without_sw = [word for word in word_tokens_test if not word in stopwords_list]
    str1 = ' '.join(tokens_without_sw)
    return str1

In [18]:
# auxiliar function to remove a pattern defined by a regular expression 
def remove_by_regex(tweet, regexp):
        return re.sub(regexp, '', tweet)

# 3 specific cleaning functions to remove numbers, url's and special characters
def remove_numbers(tweet):
    return remove_by_regex(tweet, re.compile(r"[1234567890]"))

def remove_url(tweet):
    return remove_by_regex(tweet, re.compile(r"http.?://[^\s]+[\s]?"))

def remove_special_char(tweet):
    return re.sub(r"[^a-zA-Z0-9 ]", "", tweet) #add space placeholder

# general cleaning function to do it all at once
def clean_up(tweet):
    tweet = remove_numbers(tweet)
    tweet = remove_url(tweet)
    tweet = remove_special_char(tweet)
    return tweet.lower().strip()

In [42]:
# remove stopwords
data['content'] = data['content'].apply(stopword_del)

#apply previously defined functions all at once
data["content"] = data["content"].apply(clean_up)

#stem the words in the sentences and delete too-large-whitespaces
stemmer = SnowballStemmer("english")
data["content_stemmed"] = data["content"].apply(stemmer.stem)
data["content_stemmed"] = [' '.join(x.split()) for x in data["content"]]

#convert the words in sentences to a new dataframe of vectors
vectorizer = TfidfVectorizer()
content_vect = vectorizer.fit(data.content_stemmed)
content_vect_transf = content_vect.transform(data.content_stemmed)
vector_df = pd.DataFrame.sparse.from_spmatrix(content_vect_transf.tocoo(), columns = vectorizer.get_feature_names())

In [51]:
import pickle
text_vectorizer = 'text_vectorizer_model.sav'
pickle.dump(content_vect, open(text_vectorizer, 'wb'))

In [49]:
df = pd.DataFrame.sparse.from_spmatrix(content_vect.transform(['hello how are you my friend']).tocoo(), columns = vectorizer.get_feature_names())

In [50]:
df.hello

0    0.842379
Name: hello, dtype: Sparse[float64, 0.0]

In [8]:
#add label to the vectorized dataframe
vector_df['sent_num'] = data['sent_num']

In [9]:
# create train and test sets to apply in models
train_X = vector_df.drop(columns='sent_num')[:16000]
train_y = vector_df['sent_num'][:16000]
test_X = vector_df.drop(columns='sent_num')[16000:]
test_y = vector_df['sent_num'][16000:]

# Models predictions

### Logistic Regression

In [11]:
model_LogR = LogisticRegression(max_iter = 1000, multi_class="multinomial")
model_LogR.fit(train_X, train_y)

LogisticRegression(max_iter=1000, multi_class='multinomial')

In [12]:
model_LogR.score(train_X, train_y)

0.9488125

In [13]:
model_LogR.score(test_X, test_y)

0.8605

### SupportVectorMachine

In [11]:
from sklearn.svm import LinearSVC
LSVC = LinearSVC(multi_class='crammer_singer')
LSVC.fit(train_X, train_y)

C:\Users\tolay\AppData\Local\Programs\Python\Python38-32\lib\site-packages\sklearn\svm\_base.py:976: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn("Liblinear failed to converge, increase "


LinearSVC(multi_class='crammer_singer')

In [15]:
LSVC.score(train_X, train_y)

0.98175

In [16]:
LSVC.score(test_X, test_y)

0.8965

In [12]:
emotion_detect = 'emotion_detect_model.sav'
pickle.dump(LSVC, open(emotion_detect, 'wb'))

### Random Forest

In [17]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(max_depth=200)
rfc.fit(train_X, train_y)
predictions_rfc = rfc.predict(train_X)
accuracy_score(train_y, predictions_rfc)

0.9728125

In [18]:
predictions_rfc_test = rfc.predict(test_X)
accuracy_score(test_y, predictions_rfc_test)

0.8705

In [ ]:
pipeline